In [ ]:
const axios = require("axios");

const ax = axios.create({
    baseURL: "https://www.aircourts.com/index.php/v2/api/search"
});

/**
 * Este objeto tem as keys com o nome correto dos parametros necessários
 * para introduzir no URL final para procurar campos para um determinado dia 
 * para uma certa hora.
 */ 
const requiredParams = {
    sport: 1, // Futebol de 5
    city: 12, // Cidade do Porto (código do Aircourts)
    date: null, // Data do dia que vou querer marcar o jogo
    start_time: "19:00", // Esta vai ser a hora default que vou procurar
    time_override: 1, // não sei ao certo para que serve este campo
    page: 1, // Só vai ser preciso uma página porque vou pedir resultados que chegue no campo abaixo para encher uma página
    page_size: 99, // O comentário acima está cheio de razão
    favorites: 0, // não tenho favoritos nem estou logado (em principio 😁)
};

In [ ]:
/**
 * Reminder - Se o numero for menor que 10 mais vale meter lhe um zero atrás.
 * @param {string} day 
 * @param {string} month 
 * @param {string} hour 
 * @param {string} year 
 * @param {string} minute 
 * @param {string} seconds 
 * @returns 
 */
function getDate(
  day = "02",
  month = "June" /** Tem que ser o nome em Inglês */,
  hour = "19",
  year = "2022",
  minute = "00" /** Vai chegar para ter os resultados que quero penso eu de que */,
  seconds = "00" /*  Nem importa*/
) {
  // https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Date/toLocaleDateString
  options = { weekday: "long", year: "numeric", month: "long", day: "numeric" };

  fullDate = `${month} ${day}, ${year} ${hour}:${minute}:${seconds}`;
  hoursForAircouts = `${hour}:${minute}`;
  
  date = new Date(fullDate);
  dateInCorrectTimeZone = date.toLocaleDateString("pt-EG", options);
  dateForAircourts = `${year}-${date.getMonth()+1}-${day}`
  // console.log({ fullDate, dateInCorrectTimeZone, hoursForAircouts, dateForAircourts});
  return { fullDate, dateInCorrectTimeZone, hoursForAircouts, dateForAircourts};
}
// getDate();

In [17]:
(async function () {
  try {
    // Construir url
    let time = getDate("09");
    let url = Object.keys(requiredParams)
      .map((el) /*Keys/Paramentes*/ => {
        switch (el) {
          case "date":
            requiredParams[el] = time.dateForAircourts;
            break;
          case "start_time":
            requiredParams[el] = time.hoursForAircouts;
            break;
          // The rest of the values should be correct using the default Value.
        }
        return `${el}=${requiredParams[el]}`;
      })
      .join("&");

    // console.log(url);
    todosOsCampos = await ax.get(`?${url}`);
    console.log("Recebi o resultado de ", todosOsCampos.data.total);
    todosOsCampos = todosOsCampos.data.results;

    totalDeVagas = 0
    todosOsCampos.forEach((campo, i) => {
      slots = campo.slots;

      temVagas = []
      if (slots.length > 0) {
        temVagas = slots.filter((vaga, index) => {
          if (index < 3) {
            if (vaga.locked) return false;
            else {
              if (vaga.start.endsWith(":30") || vaga.end.endsWith(":30")) {
                // console.log("❌❌❌❌ Não quero saber", `Nome do Campo: ${campo.slug}`, `Court Id: ${vaga.court_id}`, `Horário ${vaga.start} - ${vaga.end}`, "❌❌❌❌")
                return false;
              }
              console.log(`Nome do Campo: ${campo.slug}`, `Court Id: ${vaga.court_id}`, `Horário ${vaga.start} - ${vaga.end}`);
              return true; 
            }
          } else {
            return false;
          }
        })
        if (temVagas.length > 0)  {
          console.log("✅✅✅ Campo Vago \n __________________\n ",
         campo.slug, temVagas.length, campo.address, "\n __________________\n");
         totalDeVagas += temVagas.length;
        }

      }
    });
    console.log(`Há ${totalDeVagas} ${totalDeVagas > 5 ? "🥳🥳 Bota Caralho 🥳🥳" : "Fodeu 😟"}`)
  } catch (madafackingErro) {
    console.log("El erro!", madafackingErro);
  }
})();


{
  fullDate: 'June 09, 2022 19:00:00',
  dateInCorrectTimeZone: 'quinta-feira, 9 de junho de 2022',
  hoursForAircouts: '19:00',
  dateForAircourts: '2022-6-09'
}


Promise { <pending> }

Recebi o resultado de  39
Nome do Campo: footspace Court Id: 35 Horário 19:00 - 20:00
Nome do Campo: footspace Court Id: 35 Horário 20:00 - 21:00
✅✅✅ Campo Vago 
 __________________
  footspace 2 Footspace - Indoor Soccer, R. Manuel Pinto de Azevedo 269, 4100-321 Porto 
 __________________

Nome do Campo: indoor-soccer-canidelo Court Id: 192 Horário 19:00 - 20:00
✅✅✅ Campo Vago 
 __________________
  indoor-soccer-canidelo 1 R. Emissor, 4400-395 Canidelo 
 __________________

Nome do Campo: indoor-soccer-arca-d-agua Court Id: 221 Horário 21:00 - 22:00
✅✅✅ Campo Vago 
 __________________
  indoor-soccer-arca-d-agua 1 R. de Nove de Abril 912, Porto 
 __________________

Nome do Campo: indoor-soccer-madalena Court Id: 300 Horário 19:00 - 20:00
✅✅✅ Campo Vago 
 __________________
  indoor-soccer-madalena 1 R. Orfeão 376 4405-726 Madalena 
 __________________

Nome do Campo: indoor-soccer-nun-alvares Court Id: 1158 Horário 19:00 - 20:00
✅✅✅ Campo Vago 
 __________________
  indoor-soccer-nu

In [ ]:
// console.log(Object.keys(requiredParams), encodeURIComponent("19:00"))